In [68]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import itertools

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1661,tt0315824,50000000,6808550,The Big Bounce,Owen Wilson|Morgan Freeman|Gary Sinise|Sara Fo...,George Armitage,It's all in who you trust,A small-time con artist and a Hawaiian real es...,88,Comedy|Crime,Warner Bros. Pictures|Shangri-La Entertainment...,1/30/2004,4.9,2004
1535,tt0475276,15000000,76286096,United 93,J.J. Johnson|Polly Adams|Trish Gates|Nancy McD...,Paul Greengrass,"September 11, 2001. Four planes were hijacked....",A real time account of the events on United Fl...,111,Drama|History|Crime|Thriller|Action,Universal Pictures|StudioCanal|Sidney Kimmel E...,4/28/2006,6.6,2006
625,tt0942385,92000000,188072649,Tropic Thunder,Ben Stiller|Robert Downey Jr.|Jack Black|Jay B...,Ben Stiller,The movie they think they're making... isn't a...,"Vietnam veteran 'Four Leaf' Tayback's memoir, ...",107,Action|Comedy,DreamWorks SKG|Goldcrest Pictures|Red Hour Fil...,8/9/2008,6.4,2008
1272,tt2334873,18000000,97505481,Blue Jasmine,Cate Blanchett|Alec Baldwin|Sally Hawkins|Bobb...,Woody Allen,Blue Moon. I used to know the words. Now they'...,Jasmine French used to be on the top of the he...,98,Comedy|Drama,Perdido Productions,7/25/2013,6.9,2013
1307,tt2431286,12000000,100129872,Philomena,Judi Dench|Steve Coogan|Sophie Kennedy Clark|M...,Stephen Frears,These two unlikely companions are on a journey...,"A woman searches for her adult son, who was ta...",98,Drama,BBC Films|The Weinstein Company|Baby Cow Produ...,11/1/2013,7.4,2013


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {}  # Словарь для ответов

data['profit'] = data.revenue - data.budget  
# Столбец с информацей о прибыли

data.release_date = pd.to_datetime(data['release_date'])
# Переформатируем дату


def df_explode(df, col):
    """Разбивает строку по разделителю"""
    df[col] = df[col].str.split('|')
    df = df.explode(col)
    return df


def split(string):
    """Разбивает строку на список строк"""
    result = string.str.split('|')
    return result

In [5]:
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
413,tt1504320,15000000,414211549,The King's Speech,Colin Firth|Geoffrey Rush|Helena Bonham Carter...,Tom Hooper,Find your voice.,The King's Speech tells the story of the man w...,118,Drama|History,The Weinstein Company|UK Film Council|Bedlam P...,2010-09-06,7.4,2010,399211549
1377,tt0376105,30000000,93772522,Racing Stripes,Frankie Muniz|Mandy Moore|Hayden Panettiere|Br...,Frederik Du Chau,Cheer 'til you're horse!,Shattered illusions are hard to repair -- espe...,102,Comedy,Alcon Entertainment,2005-01-06,5.1,2005,63772522
432,tt1235189,8000000,3453043,Wild Target,Bill Nighy|Emily Blunt|Rupert Grint|Rupert Eve...,Jonathan Lynn,They said 'Take her out'. He got the wrong idea.,"Victor Maynard is a middle-aged, solitary assa...",98,Action|Comedy,Matador Pictures|Isle of Man Film|Magic Light ...,2010-06-17,6.1,2010,-4546957
1678,tt0486655,70000000,135560026,Stardust,Claire Danes|Charlie Cox|Michelle Pfeiffer|Rob...,Matthew Vaughn,This Summer A Star Falls. The Chase Begins.,In a countryside town bordering on a magical l...,127,Adventure|Fantasy|Romance|Family,Paramount Pictures|Ingenious Film Partners|Di ...,2007-08-09,7.0,2007,65560026
1485,tt0443543,40000000,86915017,The Illusionist,Edward Norton|Jessica Biel|Paul Giamatti|Rufus...,Neil Burger,Nothing Is What It Seems.,"With his eye on a lovely aristocrat, a gifted ...",110,Fantasy|Drama|Thriller|Romance,Michael London Productions|Stillking Films|Bob...,2006-08-18,7.0,2006,46915017


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [7]:
data1 = data.copy()
data1[data1.budget == data1.budget.max()].original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
answers['2'] = 'Gods and Generals (tt0279111)'

In [9]:
data2 = data.copy()
data2[data2.runtime == data2.runtime.max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [10]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

In [11]:
data3 = data.copy()
data3[data3.runtime == data3.runtime.min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [12]:
answers['4'] = '110'

In [13]:
data4 = data.copy()
round(data4.runtime.mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [14]:
answers['5'] = '107'

In [15]:
data5 = data.copy()
round(data5.runtime.median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [16]:
answers['6'] = 'Avatar (tt0499549)'

In [17]:
data6 = data.copy()
data6[data6.profit == data6.profit.max()].original_title

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [18]:
answers['7'] = 'The Lone Ranger (tt1210819)'

In [19]:
data7 = data.copy()
data7[data7.profit == data7.profit.min()].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [20]:
answers['8'] = '1478'

In [21]:
data8 = data.copy()
len(data8[data8.revenue > data8.budget])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [22]:
answers['9'] = 'The Dark Knight (tt0468569)'

In [23]:
data9 = data.copy()
data9[data9.release_year == 2008].sort_values(
    ['revenue'], ascending=False).head(1).original_title
# Вывод самого кассового фильма 2008 года

599    The Dark Knight
Name: original_title, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [24]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [79]:
data10 = data.copy()
data10[data10.profit == data10.profit.min()].query(
    '2012<=release_year<=2014').original_title
# Вывод самого убыточного фильма за период с 2012 по 2014 г.

1245    The Lone Ranger
Name: original_title, dtype: object

# 11. Какого жанра фильмов больше всего?

In [26]:
answers['11'] = 'Drama'

In [80]:
data11 = data.copy()
data11.genres.str.split('|').explode().value_counts().head(1).index[0]
# Разделяем столбец с жанрами, считаем их и выводим самый часто встречаемый

'Drama'

ВАРИАНТ 2

In [28]:
data11 = data.copy()
Counter(data11.genres.str.split('|').sum()).most_common(1)
# Разделяем столбец с жанрами, считаем их и выводим самый часто встречаемый

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [29]:
answers['12'] = 'Drama'

In [83]:
data12 = data.copy()
data12 = data12[data12.profit > 0].copy()  # Дф с положительной прибылью
data12.genres.str.split('|').explode().value_counts().head(1).index[0]
# Разделяем столбец с жанрами, сортируем по частоте, выводим первый заголовок

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [31]:
answers['13'] = 'Peter Jackson'

In [85]:
director_revenue = data.copy()

director_revenue = df_explode(director_revenue, 'director')
# Разбиваем информацию в столбце director на отдельные строки

director_revenue.groupby(['director'])['revenue'].agg(['sum']).sort_values(
        'sum', ascending=False).head(1)
# Группируем суммарные кассовые сборы по режисерам

,sum
director,
Peter Jackson,6490593685


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [33]:
answers['14'] = 'Robert Rodriguez'

In [34]:
data14 = data.copy()
data14[data14.genres.str.contains('Action')].director.str.split('|').explode(
    'director').value_counts(ascending=False).head(1).index[0]
# Делаем запрос к Дф о строках содержащих Action в столбце genres,
# и считаем количество значений соотнесенных с разбитым столбцом director

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [35]:
answers['15'] = 'Chris Hemsworth'

In [89]:
data2012 = data[data.release_year == 2012].copy()  
# Дф с информацией за 2012 год

data2012 = df_explode(data2012, 'cast')
# Разбиваем информацию в столбце cast на отдельные строки

data2012.groupby(['cast'])['revenue'].sum().sort_values(
    ascending=False).head(1).index[0]
# Группируем актеров и сборы, выводим у кого из актеров сумма сборов выше

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [37]:
answers['16'] = 'Matt Damon'

In [88]:
cast_revenue = data[data.budget > data.budget.mean()].copy()
# Создаем Дф, где бюджет выше среднего

cast_revenue = df_explode(cast_revenue, 'cast')
# Разбиваем информацию в столбце cast на отдельные строки

cast_revenue.groupby(['cast'])['imdb_id'].count().sort_values(
    ascending=False).head(1).index[0]
# Группируем актеров и фильмы, выводим у кого из актеров больше фильмов

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [39]:
answers['17'] = 'Action'

In [91]:
data_cage = data.copy()

data_cage = df_explode(data_cage, 'cast')
data_cage = df_explode(data_cage, 'genres')
# Разбиваем информацию в столбцах cast и genres на отдельные строки

data_cage = data_cage[data_cage.cast == 'Nicolas Cage']
# Фильтруем Дф чтобы осталась информация только по Николасу Кейджу

data_cage.genres.value_counts(ascending=False).head(1).index[0]
# Группируем Дф по жанрам

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [41]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

In [42]:
data_par = data.copy()
# Создаем копию Дф

data_par = df_explode(data_par, 'production_companies')
# Разбиваем информацию о киностудиях на отдельные строки

data_par = data_par[data_par.production_companies == 'Paramount Pictures']
# Фильтруем, чтобы осталась только Paramount Pictures

data_par[data_par.profit == data_par.profit.min()].original_title
# Смотрим какой фильм имеет самую маленькую прибыль

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [43]:
answers['19'] = '2015'

In [44]:
data19 = data.copy()
data19.groupby(['release_year'])['revenue'].sum().sort_values(
    ascending=False).head(1).index[0]
# Группируем запрос по датам релиза и сумме кассовых сборов

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [45]:
answers['20'] = '2014'

In [46]:
data_WB = data.copy()
# Делаем копию Дф

data_WB = df_explode(data_WB, 'production_companies')
# Разбиваем информацию о киностудиях на отдельные строки

data_WB = data_WB[data_WB.production_companies.str.contains('Warner Bros')]
# Создаем Дф с информацией только о компаниях, имеющих в названии Warner Bros

data_WB.groupby(['release_year'])['profit'].sum().sort_values(    
    ascending=False).head(1).index[0]
# Группируем года выхода и суммарную прибыль по возрастанию

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [47]:
answers['21'] = 'Сентябрь'

In [48]:
data21 = data.copy()
data21.release_date.dt.month.value_counts().index[0]
# Выделяем месяц из столбца с датой

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [49]:
answers['22'] = '450'

In [50]:
data22 = data.copy()
len(data22[data22.release_date.dt.month.isin([6, 7, 8,])])
# Выделяем из даты месяц, фильтруем по летним месяцам и считаем длину

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [51]:
answers['23'] = 'Peter Jackson'

In [52]:
data_director = data.copy()

data_director = df_explode(data_director, 'director')
# Разбиваем информацию о режиссерах на отдельные строки

data_director = data_director[data_director.release_date.dt.month.isin(
    ['1', '2', '12'])]
# Выделяем из Дф информацию только по зимним месяцам

data_director.director.value_counts(ascending=False).head(1).index[0]
# Выводим кто из режиссеров чаще встречается

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [53]:
answers['24'] = 'Four By Two Productions'

In [54]:
data_str = data.copy()

data_str['title_length'] = data_str['original_title'].str.len()
# Создадим столбец с длиной названия

companies = data_str['production_companies'].str.split('|').explode().unique()
# Получим список уникальных названий компаний

sum_gen = pd.Series(dtype='float64', index=companies)
# Создадим серию для хранения средних значений


for comp in companies:
    sum_gen[comp] = data_str['title_length'][data_str['production_companies'].
                          map(lambda x: True if comp in x else False)].mean()
# Вычислим среднее значение для всех названий фильмов, созданных компанией

    
sum_gen.sort_values(ascending=False).head(1).index[0]
# Выведем компанию с самыми большими значениями

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [55]:
answers['25'] = 'Midnight Picture Show'

In [56]:
data_str2 = data.copy()

data_str2['overview_length'] = data_str2['overview'].str.split(' ').str.len()
# Cоздадим столбец с длиной описаний

data_str2 = df_explode(data_str2, 'production_companies')
# Разбиваем информацию о киностудиях на отдельные строки

data_str2.groupby('production_companies')['overview_length'].mean().idxmax()
# Группируем студии и средние длины описаний, выводим студию с макс. значением

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 

In [65]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

In [64]:
data26 = data.copy()

one_procent = np.quantile(data.vote_average, 0.99)
# Записываем в переменную 100-ый квантиль средних оценок

data26[data26.vote_average >= one_procent].sort_values(
    by='vote_average', ascending=False)['original_title']
# Выводим все фильмы с оценкой выше или равной квантилю

599                                       The Dark Knight
9                                              Inside Out
34                                                   Room
118                                          Interstellar
125                                    The Imitation Game
370                                             Inception
1191                                     12 Years a Slave
1183                              The Wolf of Wall Street
1081        The Lord of the Rings: The Return of the King
872                                           The Pianist
1800                                              Memento
119                               Guardians of the Galaxy
138                              The Grand Budapest Hotel
128                                             Gone Girl
283                                            Mr. Nobody
1688                                  There Will Be Blood
1563                Eternal Sunshine of the Spotless Mind
1444          

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [76]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

In [75]:
data_cast = data.copy()
pairs = []  # Список для пар актеров

data_cast['cast'] = split(data_cast['cast'])
# Разобьем информацию в столбце cast

for i in range(0, len(data_cast)):
    for item in itertools.combinations(data_cast['cast'][i], 2):
        pairs.append(item)
# Перебираем все пары актеров, записываем их в список, 
# указывая количество повторений        
        
Counter(pairs).most_common(1)
# Выводим наиболее часто встречаемые

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

# Submission

In [77]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '110',
 '5': '107',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [62]:
# и убедиться, что ничего не пропустил)
len(answers)

27